# Workspace cleanup

In [ ]:
%cd /content
! rm -rf Yet* datasets weights logs experiments *.pth

# Setup workspace

In [ ]:
! git clone https://github.com/grafiszti/Yet-Another-EfficientDet-Pytorch.git

%cd /content/Yet-Another-EfficientDet-Pytorch
! git checkout package_making
! pip install -r requirements.txt

%cd /content

! mkdir weights
! wget –q https://github.com/zylo117/Yet-Another-Efficient-Pytorch/releases/download/1.0/efficientdet-d{0..8}.pth
! mv *.pth weights
! mkdir datasets
! mkdir experiments

# Utils function to upload trained model checkpoint to gdrive

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os

def authenticate_user():
    auth.authenticate_user()
    drive_service = build('drive', 'v3')
    return drive_service

def upload_last_model_to_drive(training_dir: str):
    drive_service = authenticate_user()
    filepath = training_dir + "/" + sorted(os.listdir(training_dir))[-2]

    print(f"File to export: {filepath}")

    file_metadata = {
        'name': os.path.basename(filepath), 
        'mimeType': 'application/octet-stream'
    }
    media = MediaFileUpload(filepath, mimetype='application/octet-stream', resumable=True)

    created = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()

    print('File ID: {}'.format(created.get('id')))

# Create file with project config 
Zip files should contain dataset in COCO format. 

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

project_name = "test_coco_dataset"

datasets_to_download = [
  (f"{project_name}.zip", "1mxqzmJLeI7iRlHsVNoRnSNAYAZsOoNkP")
]

for filename, id in datasets_to_download:
    GoogleDriveDownloader.download_file_from_google_drive(
        dest_path=f"/content/datasets/{filename}", 
        file_id=id,
        unzip=True, 
        showsize=True,
        overwrite=True,
    )
    
project_config=f"""
project_name: {project_name}  # also the folder name of the dataset that under data_path folder
train_set: train
val_set: val
num_gpus: 1

# mean and std in RGB order, actually this part should remain unchanged as long as your dataset is similar to coco.
mean: [0.485, 0.456, 0.406]
std: [0.229, 0.224, 0.225]

# this is coco anchors, change it if necessary
anchors_scales: '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios: '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]'

# must match your dataset's category_id. category_id is one_indexed.
obj_list: ["car"]
"""

with open(f"{project_name}.yml", "w") as f: 
  f.write(project_config)

# Run training
Change th `PROJECT_NAME` to your project name.

In [ ]:
! python Yet-Another-EfficientDet-Pytorch/train.py \
    -p PROJECT_NAME.yml \
    -c 2 \
    --batch_size 2 \
    --optim adamw \
    --lr 0.0001 \
    --num_epochs 1 \
    -w weights/efficientdet-d0.pth \
    --log_path experiments/logs_d0_epochs1_batch_2_lr0.0001_headonlyFalse \
    --saved_path experiments/logs_d0_epochs1_batch_2_lr0.0001_headonlyFalse \
    --head_only False